In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split, cross_val_score


## Loading The data

In [ ]:
df = pd.read_csv('../input/summer-products-and-sales-in-ecommerce-wish/summer-products-with-rating-and-performance_2020-08.csv')
df.head()

## Some Basic Data Exploration

In [ ]:
df.head().T

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isna().sum()

## Dropping Some Columns
Dropping the columns with many unique categories coz they wont be very helpful and they will just add many dimensions to the data.

In [ ]:
max_unique = 60
high_unique = [col for col in df.select_dtypes(exclude=np.number)
                   if df[col].nunique() > max_unique]
df = df.drop(columns=high_unique)
df.info()

In [ ]:
df.head().T

In [ ]:
df['currency_buyer'].value_counts()

## Building correlation

In [ ]:
corr = df.corr()
corr

In [ ]:
fig,ax = plt.subplots(figsize = (16,16))
ax = sns.heatmap(corr,
                 annot=True,
                 linewidths=1.2,
                 fmt=".2f",
                 cmap="YlGnBu");

## Visualizing some columns

In [ ]:
sns.countplot(df['origin_country']);

In [ ]:
sns.countplot(df['urgency_text']);

In [ ]:
sns.barplot(x = df.origin_country,y = df.units_sold);

In [ ]:
df['has_urgency_banner'].value_counts()

Dropping some more columns

In [ ]:
df = df.drop(['crawl_month','origin_country','rating_count','shipping_option_name','urgency_text'],axis = 1)

In [ ]:
df.head().T

In [ ]:
df.info()

# LabelEncoding 'currency_buyer'& 'theme' columns

In [ ]:
le = LabelEncoder()
df['currency_buyer'] = le.fit_transform(df['currency_buyer'])
df['theme'] = le.fit_transform(df['theme'])

Checking all the columns with Missing values

In [ ]:
for label,content in df.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isna(content).sum():
            print(label)

In [ ]:
df['has_urgency_banner'].value_counts()

filling 'has_urgency_banner' column with 0 

In [ ]:
df['has_urgency_banner'] = df['has_urgency_banner'].fillna(0)

filling all the other columns with meadian

In [ ]:
for label,content in df.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isna(content).sum():
            df[label] = content.fillna(content.median())

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.isna().sum()

# Splitting the data into Training and Testing sets

In [ ]:
x = df.drop('units_sold',axis = 1)
y = df['units_sold']

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y, test_size = 0.2)

# Building the models
# RandomForest

In [ ]:
model = RandomForestRegressor(n_estimators = 1000,random_state = 42)
model.fit(x_train,y_train)

In [ ]:
model.score(x_test,y_test)

# LinearRegression

In [ ]:
model1 = LinearRegression()
model1.fit(x_train,y_train)

In [ ]:
model1.score(x_test,y_test)

In [ ]:
scores = pd.DataFrame({'RandomForest': model.score(x_test,y_test),
                       'LinearRegression': model1.score(x_test,y_test)},
                        index = [0])


# Plotting the scores of both the model

In [ ]:
scores.T.plot(kind = 'bar',
              figsize = (10,10))
plt.title('Scores of all Model')
plt.xlabel('Model Name')
plt.ylabel('Scores');